# 箭头函数

In [ ]:
hello = function(){
        return 'hello world'
    }
// 将变成
hello = () => {
    return 'hello world'
}
// 最终变成
hello = () => 'hello world'

##### 箭头函数没有自己的 this， 如果调用 this， 那就会向外找。

# 回调函数：

作为某函数的参数的函数叫做回调函数 例：log1就是回调函数

In [ ]:
const log1 = () => {
    console.log(1);
};
setTimeout(log1, 1000)

// 通常写作：log1写成匿名函数
setTimeout(() => {
        console.log(1);
    }, 1000);

# 异步函数：单核运行多线程

传统异步是回调函数

In [ ]:
setTimeout(() => {
    console.log('等3秒');

    setTimeout(() => {
        console.log('再等3秒');

        setTimeout(() => {
            console.log('又等3秒');

            //...
        }, 3000);
    }, 3000);
}, 3000);
// 此方法会不断向右延申， 影响阅读性， 又称回调地狱

新异步： Promise

In [ ]:
// 返回一个 Promise 对象， 承诺在未来某个时刻返回数据
fetch("https://jsonplaceholder.typicode.com/posts/1") 
    .then( (response) => {    // 用 then 方法来传递一个回调函数
        // ...
    });
// 如果这个请求在未来成功完成， 那么回调函数会被调起， 请求的结果也会以参数的形式传递进来

Promise 和回调函数的区别：

可以用链式结构将多个异步操作串联起来

In [ ]:
fetch("https://jsonplaceholder.typicode.com/posts/1")
    .then( (response) => response.json());
// 这里的response.json()方法也会返回一个Promise
// 代表在将来的某个时刻将返回的数据转换成 JSON 格式

Then: 如果想等到此程序完成后再执行其他的操作， 可在后边追加一个 then 

In [ ]:
fetch("https://jsonplaceholder.typicode.com/posts/1")
    .then( (response) => response.json())
    .then((json) => console.log(json))  // 将结果打印出来
    .then( )
    .then( )
    .then( );

// 避免了层层嵌套

Catch: 若执行中报错

In [ ]:
fetch("https://jsonplaceholder.typicode.com/posts/1")
    .then( (response) => response.json())
    .then((json) => console.log(json))  // 将结果打印出来
    
    .catch((error) => {
        console.error(error);
    }) // 任意的 then 如果出错就会报错，并且后续的 then 不会被执行
    
    .finally(() => {
        stopLoadingAnimation();
    }); 
    // 可在此进行清理工作， 比如用到了加载动画， 可在 finally 中关闭它。

Finally: 在 Promise 链结束后调用， 无论失败与否。

# Async & Await

在 Promise 的基础上， 让异步操作更加的简洁明了

Await: 代替 then, 在异步函数中使用异步函数

* await 虽然看上去会暂停函数的执行， 但在等待的过程中， JavaScript 同样可以处理其他的任务。
* await 的底层是基于 Promise 和事件循环机制实现的: Promise + 事件循环（Event Loop）

In [ ]:
// 使用 async 关键字将函数标记为异步函数
async function f(){
    const response = await fetch("http://...");  // 可调用其他异步函数，使用 await
    const json = await response.json();
    console.log(json);
}

f();

Await 陷阱 1

In [ ]:
// 错误例子！！！！
async function f(){
    const a = await fetch("http://.../post/1");
    const b = await fetch("http://.../post/2");
}
f()
// 如果分别去 await 这两个异步操作， 
// 会打破两个 fetch() 操作的并行， 因为 a 执行完才会执行 b。

In [ ]:
// 正确写法：
async function f(){
    const promiseA = await fetch("http://.../post/1");
    const promiseB = await fetch("http://.../post/2");

    const [a, b] = await Promise.all([promiseA, promiseB]);
    // 先用 Promise.all 组合起来， 再 await。
}

f();

Await 陷阱 2

In [ ]:
// 错误例子！！！！不可以用 forEach() / map()
async function f(){
    [1, 2, 3].forEach(async (i) => {
        await someAsyncOperation();
    });

    console.log("done");
}
f();
// 尽管写了 await 但是 forEach 会立刻返回， 他不会等到所有异步操作执行完毕
// 如果我们希望等待循环中的异步操作都一一完成之后才继续执行， 应用 for 循环

In [ ]:
// 正确例子：
async function f(){
    for (let i of [1, 2, 3]) {
        await someAsyncOperation();
    }

    console.log("done");
}
f();

In [ ]:
// 若想要循环中的所有操作都并发执行
// 正确例子：
async function f(){
    const promises = [
        someAsyncOperation(),
        someAsyncOperation(),
        someAsyncOperation(),
    ];

    for await (let result of promises){

    }
    console.log('done');
}

Await 陷阱 3

In [ ]:
// 错误例子！！！
await someAsyncOperaton();
// 不能在全局或普通函数里使用 await 关键字
// 若想在最外层使用 await， 那么需要先定义一个异步函数

In [ ]:
// 正确例子：
async function f(){
    await someAsyncOperation();
}
f();
// 简洁写法
(async () => {
    await someAsyncOperation();
}) ();

例子：

使用 async 和 await 可以几乎不再需要使用底层的 Promise 对象， 包括调用它的 then(), catch() 函数等。

In [ ]:
// 不再需要这样写， 但是正确写法我也不知道。
function requestSomeData(method, url) {
    return new Promise(function (resolve, reject) {
        var xhr = new XMLHttpRequest();
        xhr.open(method, url);
        xhr.onload = function () {
            if (this.status >= 200 && this.status < 300) {
                resolve(xhr.response);
            } else {
                reject({
                    status: this.status,
                    statusText: xhr.statusText,
                });
            }
        };
        xhr.onerror = function () {
            reject({
                status: this.status,
                statusText: xhr.statusText,
            });
        }
    })
}